## Model: United-atom Model 

In [1]:
from os import path, system
from shutil import copyfile
import datetime
import time
import scipy.constants as constants
import pandas as pd
import numpy as np
import enm, prm, ic_table

## Calculate the variance of each bond through normal mode analysis(NMA)

In [2]:
rootdir = '/Users/yizao/PycharmProjects/ENM'
host = 'pnas_amber_clean'
type_na = 'bdna+bdna'
nadir = path.join(rootdir, host, type_na)
agent = enm.ENMAgent(host, type_na)

/Users/yizao/PycharmProjects/ENM/pnas_amber_clean exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/input exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/charmm_inp exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/charmm_dat exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/mode_traj exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/ic exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/ic_fluct_mat exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/rtf_ic_str exists
/Users/yizao/PycharmProjects/ENM/pnas_amber_clean/bdna+bdna/data exists


### Part 0-1: Get initial na.avg.ic and na.fluct.ic
IC Fluctuate: Make ic/mode.0.ic  
vim pnas_amber_clean/bdna+bdna/charmm_inp/ic_fluct.inp  
charmm_yz < pnas_amber_clean/bdna+bdna/charmm_inp/ic_fluct.inp > pnas_amber_clean/bdna+bdna/charmm_dat/ic_fluct.dat  
cat < pnas_amber_clean/bdna+bdna/ic/mode.0.ic | sed 's/-99 / -99 /' > pnas_amber_clean/bdna+bdna/data/na.fluct.ic  
  
IC Average: make ic/mode.0.avg.ic  
vim pnas_amber_clean/bdna+bdna/charmm_inp/ic_avg.inp  
charmm_yz < pnas_amber_clean/bdna+bdna/charmm_inp/ic_avg.inp > pnas_amber_clean/bdna+bdna/charmm_dat/ic_avg.dat  
cat < pnas_amber_clean/bdna+bdna/ic/mode.0.avg.ic | sed 's/-99 / -99 /' > pnas_amber_clean/bdna+bdna/data/na.avg.ic   

In [3]:
# Read icavg
avg_file = path.join(nadir, 'data', 'na.avg.ic')
icavg_0 = ic_table.ICTable(avg_file, initial=True)
#print(icavg.values)

# Read icfluct
fluct_file = path.join(nadir, 'data', 'na.fluct.ic')
icfluct_0 = ic_table.ICTable(fluct_file, initial=True)
#print(icfluct.values)

### Part 0-2: Get the initial equilibrium distance and force constant and write PRM

In [4]:
# Initialized Constants
T = 310.0 # temperature, 300 K

# RT kcal/mol # https://en.wikipedia.org/wiki/KT_(energy)
RT = T * (constants.k * constants.N_A / (constants.calorie * constants.kilo))

b_0 = icavg_0.values  # Initial Guess of equilibrium bond length
k_0 = RT / np.square(icfluct_0.values) # Initial Guess of force constants

kbpair_0 = ic_table.KBPair(read_from_prm=False, icavg=icavg_0, icfluct=icfluct_0, rt=RT)

In [5]:
kbpair_0.convert_to_df().head()

,b,k,name1,name2
0,9.5684,3.373947,A25,A1
1,10.1299,2.173341,A25,A2
2,10.0924,1.462113,A25,A3
3,8.1324,3.080353,A25,A4
4,7.5778,4.052263,A25,A5


#### Warning!!! Only restart from the zero iteration

In [6]:
f_out = path.join('.', host, type_na, 'data', 'na_enm.prm')
prm_agent = prm.PRM(host, type_na, kbpair_0, iternum=0)
prm_agent.write_prm(f_out)

### Part 1: Get IC average and IC fluctuation

vim pnas_amber_clean/bdna+bdna/charmm_inp/nmainit.inp  
charmm_yz < pnas_amber_clean/bdna+bdna/charmm_inp/nmainit.inp > pnas_amber_clean/bdna+bdna/charmm_dat/nmainit.dat 

### Part 2: Read IC Table

In [7]:
# Read icavg
avg_file = path.join(nadir, 'data', 'average.ic')
icavg = ic_table.ICTable(avg_file)
#print(icavg.values)

# Read icfluct
fluct_file = path.join(nadir, 'data', 'fluct.ic')
icfluct = ic_table.ICTable(fluct_file)
#print(icfluct.values)

# Initialized Constants
T = 310.0 # temperature, 310 K

# RT kcal/mol # https://en.wikipedia.org/wiki/KT_(energy)
RT = T * (constants.k * constants.N_A / (constants.calorie * constants.kilo))

kbpair = ic_table.KBPair(read_from_prm=False, icavg=icavg, icfluct=icfluct, rt=RT)

In [8]:
kbpair.convert_to_df().head()

,b,k,name1,name2
0,9.707712,103.756110,A25,A1
1,10.297383,50.485934,A25,A2
2,10.201489,48.378769,A25,A3
3,8.189096,112.509080,A25,A4
4,7.660487,135.579510,A25,A5


### Part 3: Fluctuation-Matching (Do NMA and iterate)

vim pnas_amber_clean/bdna+bdna/charmm_inp/nma.inp  
charmm_yz<pnas_amber_clean/bdna+bdna/charmm_inp/nma.inp> pnas_amber_clean/arna+arna/charmm_dat/nma.dat

In [9]:
T = 310.0 # temperature, 300 K

# RT kcal/mol # https://en.wikipedia.org/wiki/KT_(energy)
RT = T * (constants.k * constants.N_A / (constants.calorie * constants.kilo))

# Learning Rate
alpha = RT * 0.02

charmm_exec = '/Users/yizao/c41b1_yz/exec/osx/charmm'

In [10]:
start = 0
end = 400  # Temp

k = kbpair_0.d['k']
target = np.reciprocal(np.square(icfluct_0.values))

avg_file = path.join(nadir, 'data', 'average.ic')
fluct_file = path.join(nadir, 'data', 'fluct.ic')
prmfile = path.join('.', host, type_na, 'data', 'na_enm.prm')
prm_backup = path.join('.', host, type_na, 'data', 'backup', 'na_enm.backup.prm')
prm_backup_0 = path.join('.', host, type_na, 'data', 'backup', 'na_enm.backup.0.prm')
err_file = path.join('.', host, type_na, 'data', 'error.txt')

copyfile(prmfile, prm_backup_0)

f = open(err_file, 'w')
f.write('Created at {0}\n'.format(datetime.datetime.now()))
f.write('{0:<5} {1:8}\n'.format('n_iter', 'error'))
f.close()

for i in range(start, end):
    print("IterNum: {0}".format(i))
    # do NMA
    cmd = '{0}<./{1}/{2}/charmm_inp/nma.inp> ./{1}/{2}/charmm_dat/nma.dat'.format(charmm_exec, host, type_na)
    system(cmd)
    time.sleep(3)

    # Read icavg
    new_icavg = ic_table.ICTable(avg_file)

    # Read icfluct
    new_icfluct = ic_table.ICTable(fluct_file)
    optimized = np.reciprocal(np.square(new_icfluct.values))

    new_kbpair = ic_table.KBPair(read_from_prm=False, icavg=new_icavg, icfluct=new_icfluct, rt=RT)

    # Update k
    k -= alpha * (optimized - target)
    k[np.where(k < 0.)] = 0. # Make all negative values zero
    new_kbpair.d['k'] = k

    # Calculate RMSD error and Write
    error = icfluct_0.values - new_icfluct.values
    error = np.square(error).mean()
    error = np.sqrt(error)
    f = open(err_file, 'a')
    f.write('{0:<5} {1:8.4f}\n'.format(i+1, error))
    f.close()
    
    # Backup PRM
    copyfile(prmfile, prm_backup)
    
    # Update PRM
    prm_agent = prm.PRM(host, type_na, new_kbpair, iternum=i)
    prm_agent.write_prm(prmfile)

IterNum: 0
IterNum: 1
IterNum: 2
IterNum: 3
IterNum: 4
IterNum: 5
IterNum: 6
IterNum: 7
IterNum: 8
IterNum: 9
IterNum: 10
IterNum: 11
IterNum: 12
IterNum: 13
IterNum: 14
IterNum: 15
IterNum: 16
IterNum: 17
IterNum: 18
IterNum: 19
IterNum: 20
IterNum: 21
IterNum: 22
IterNum: 23
IterNum: 24
IterNum: 25
IterNum: 26
IterNum: 27
IterNum: 28
IterNum: 29
IterNum: 30
IterNum: 31
IterNum: 32
IterNum: 33
IterNum: 34
IterNum: 35
IterNum: 36
IterNum: 37
IterNum: 38
IterNum: 39
IterNum: 40
IterNum: 41
IterNum: 42
IterNum: 43
IterNum: 44
IterNum: 45
IterNum: 46
IterNum: 47
IterNum: 48
IterNum: 49
IterNum: 50
IterNum: 51
IterNum: 52
IterNum: 53
IterNum: 54
IterNum: 55
IterNum: 56
IterNum: 57
IterNum: 58
IterNum: 59
IterNum: 60
IterNum: 61
IterNum: 62
IterNum: 63
IterNum: 64
IterNum: 65
IterNum: 66
IterNum: 67
IterNum: 68
IterNum: 69
IterNum: 70
IterNum: 71
IterNum: 72
IterNum: 73
IterNum: 74
IterNum: 75
IterNum: 76
IterNum: 77
IterNum: 78
IterNum: 79
IterNum: 80
IterNum: 81
IterNum: 82
IterNum: 83
It

### Part 4: Restart

In [7]:
start = 80
end = 400

k = kbpair_0.d['k']
target = np.reciprocal(np.square(icfluct_0.values))

avg_file = path.join(nadir, 'data', 'average.ic')
fluct_file = path.join(nadir, 'data', 'fluct.ic')
prmfile = path.join('.', host, type_na, 'data', 'na_enm.prm')
prm_backup = path.join('.', host, type_na, 'data', 'backup', 'na_enm.backup.prm')
err_file = path.join('.', host, type_na, 'data', 'error.txt')

for i in range(start, end):
    print("IterNum: {0}".format(i))
    # do NMA
    cmd = '{0}<./pnas/arna+arna/charmm_inp/nma.inp> ./pnas/arna+arna/charmm_dat/nma.dat'.format(charmm_exec)
    system(cmd)
    time.sleep(3)

    # Read icavg
    new_icavg = ic_table.ICTable(avg_file)

    # Read icfluct
    new_icfluct = ic_table.ICTable(fluct_file)
    optimized = np.reciprocal(np.square(new_icfluct.values))

    new_kbpair = ic_table.KBPair(read_from_prm=False, icavg=new_icavg, icfluct=new_icfluct, rt=RT)

    # Update k
    k -= alpha * (optimized - target)
    k[np.where(k < 0.)] = 0. # Make all negative values zero
    new_kbpair.d['k'] = k

    # Calculate RMSD error and Write
    error = icfluct_0.values - new_icfluct.values
    error = np.square(error).mean()
    error = np.sqrt(error)
    f = open(err_file, 'a')
    f.write('{0:<5} {1:8.4f}\n'.format(i+1, error))
    f.close()
    
    # Backup PRM
    copyfile(prmfile, prm_backup)
    
    # Update PRM
    prm_agent = prm.PRM(host, type_na, new_kbpair, iternum=i)
    prm_agent.write_prm(prmfile)

IterNum: 80
IterNum: 81
IterNum: 82
IterNum: 83
IterNum: 84
IterNum: 85
IterNum: 86
IterNum: 87
IterNum: 88
IterNum: 89
IterNum: 90
IterNum: 91
IterNum: 92
IterNum: 93
IterNum: 94
IterNum: 95
IterNum: 96
IterNum: 97
IterNum: 98
IterNum: 99
IterNum: 100
IterNum: 101
IterNum: 102
IterNum: 103
IterNum: 104
IterNum: 105
IterNum: 106
IterNum: 107
IterNum: 108
IterNum: 109
IterNum: 110
IterNum: 111
IterNum: 112
IterNum: 113
IterNum: 114
IterNum: 115
IterNum: 116
IterNum: 117
IterNum: 118
IterNum: 119
IterNum: 120
IterNum: 121
IterNum: 122
IterNum: 123
IterNum: 124
IterNum: 125
IterNum: 126
IterNum: 127
IterNum: 128
IterNum: 129
IterNum: 130
IterNum: 131
IterNum: 132
IterNum: 133
IterNum: 134
IterNum: 135
IterNum: 136
IterNum: 137
IterNum: 138
IterNum: 139
IterNum: 140
IterNum: 141
IterNum: 142
IterNum: 143
IterNum: 144
IterNum: 145
IterNum: 146
IterNum: 147
IterNum: 148
IterNum: 149
IterNum: 150
IterNum: 151
IterNum: 152
IterNum: 153
IterNum: 154
IterNum: 155
IterNum: 156
IterNum: 157
IterNu

### Part 5: Get a minimized structure accdoring to converged parameter file(prm)

vim ./pnas_amber_clean/bdna+bdna/charmm_inp/get_minim_after_fluct.inp  
charmm_yz < ./pnas_amber_clean/bdna+bdna/charmm_inp/get_minim_after_fluct.inp> ./pnas_amber_clean/bdna+bdna/charmm_dat/get_minim_after_fluct.dat  
vmd -cor ./pnas_amber_clean/bdna+bdna/data/minim_after_fm.crd  